<a href="https://colab.research.google.com/github/MCanela-1954/Data/blob/main/Notebooks/%5BRAG-9.2%5D%20QA%20on%20IESE%20Business%20School%20(short%20version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [RAG-9.2] Q/A on IESE Business School (short version)

## Setup

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig, EmbedContentConfig
import numpy as np, pandas as pd
client = genai.Client(api_key = 'GOOGLE_API_KEY')

## Model selection

In [ ]:
chat_model = 'gemini-2.0-flash'
embed_model = 'models/text-embedding-004'
embed_config = EmbedContentConfig(task_type = 'SEMANTIC_SIMILARITY', output_dimensionality = 768)

## Importing the QA data

In [ ]:
path = 'https://raw.githubusercontent.com/IAlegre-MCanela/Summer_Course/main/Session%209/'
data = pd.read_json(path + 'faq.json')

## Importing the vector database

In [ ]:
embeds = pd.read_csv(path + 'embeds.csv', header = None)

## System guidelines #


In [ ]:
guidelines = '''
You are a chatbot responding queries about MBA programs at IESE Business School.
You will be shown the query.
Make your response shorter than 50 words.
Use only the information provided.
If the information provided doesn't mention the subject of the query, limit your response to the following:
<response>
Unfortunately, I don't have information about this subject. Please, directly contact with the school.
</response>
'''

## Chat function

In [ ]:
def chat(query):
	result = client.models.embed_content(config = embed_config, model = embed_model, contents = [query])
	query_embed = result.embeddings[0].values
	query_embed = np.array(query_embed)
	data['similarity'] = np.dot(embeds, query_embed.T)
	top5 = data.sort_values(by='similarity', ascending=False).head(5)
	info = 'Information:\n' + top5['answer'].sum()
	chat_config = GenerateContentConfig(system_instruction = guidelines + info)
	response = client.models.generate_content(config = chat_config, contents = query, model = chat_model)
	print(response.text)

In [ ]:
chat('Do you control class attendance?')

In [ ]:
chat('Are there elective courses in the Executive MBA?')